In [ ]:
import os
import matplotlib.pyplot as plt

# Specify the directory containing log files
log_dir = "problem1"

# Initialize lists to store Bitrate and Cwnd values for each log file
bitrates_list = []
cwnds_list = []

# Specify colors for each log file
colors = ['b', 'g', 'r', 'c', 'k', 'm', 'y' , 'brown']  # You can add more colors as needed
linestyles = ['-', '--', '-.', ':']  # Different linestyles for "less" files
filenames = []

# Loop through all files in the log directory
for i, filename in enumerate(os.listdir(log_dir)):
    if filename.endswith("_results.log"):
        filenames.append(filename[:-12])
        
        # Initialize lists for this log file
        bitrates = []
        cwnds = []

        log_file = os.path.join(log_dir, filename)

        # Read data from the log file
        with open(log_file, "r") as file:
            for line in file:
                if line.startswith("Bitrate:") and "Cwnd:" in line:
                    parts = line.split()
                    bitrate = float(parts[1])
                    if parts[4] == 'KBytes':
                        cwnd = float(parts[3]) / 1024
                    else:
                        cwnd = float(parts[3])
                    bitrates.append(bitrate)
                    cwnds.append(cwnd)

        # Append the data for this log file to the lists
        bitrates_list.append(bitrates)
        cwnds_list.append(cwnds)

# Create a single figure for all log files
plt.figure(figsize=(8, 6))

# Plot each log file's data with a different color and linestyle based on filename
for i, (bitrates, cwnds) in enumerate(zip(bitrates_list, cwnds_list)):
    color = colors[i % len(colors)]  # Cycle through colors
    linestyle = 'dashed' if "less" not in filenames[i] else 'solid'
    label = filenames[i]  # Set the label as the filename without extension
    plt.scatter(cwnds, bitrates, marker='o', color=color, label=label)
    plt.plot(cwnds, bitrates, linestyle=linestyle, color=color)

plt.xlabel('Congestion Window Size (MBytes)')
plt.ylabel('Thourghput (Mbps)')
plt.title('Thourghput vs. Congestion Window Size')
plt.grid(True)
plt.legend()

# Show or save the plot (you can choose to save it if needed)
plt.show()


In [ ]:
import json
import os

# Get the current directory
current_directory = os.getcwd()

# List all files in the current directory
all_files = os.listdir(current_directory)

# Filter JSON files
json_files = [filename for filename in all_files if filename.endswith('.json')]

# Iterate through JSON files
for input_filename in json_files:
    # Create the corresponding output log file name
    output_filename = os.path.splitext(input_filename)[0] + '.log'

    try:
        with open(input_filename, 'r') as json_file, open(output_filename, 'w') as log_file:
            data = json.load(json_file)

            # Extract the intervals section
            intervals = data.get("intervals", [])

            # Iterate through intervals
            for interval in intervals:
                # Iterate through streams within each interval
                for stream in interval.get("streams", []):
                    # Extract RTT and snd_cwnd from the stream
                    rtt = stream.get("rtt", None)
                    snd_cwnd = stream.get("snd_cwnd", None)

                    # Write RTT and snd_cwnd to the log file if available
                    if rtt is not None and snd_cwnd is not None:
                        log_file.write(f"RTT: {rtt}, snd_cwnd: {snd_cwnd}\n")

            print(f"Data from '{input_filename}' has been extracted and saved to '{output_filename}'")

    except Exception as e:
        print(f"An error occurred while processing '{input_filename}': {str(e)}")


# Problem2
plot delay vs congestion window size

In [ ]:
import os
import matplotlib.pyplot as plt

# Specify the directory containing log files
log_dir = "problem2"

# Initialize lists to store RTT and snd_cwnd values for each log file
rtt_list = []
snd_cwnd_list = []

# Specify colors for each log file
colors = ['b', 'g', 'r', 'c', 'k', 'm', 'y', 'brown']  # You can add more colors as needed
linestyles = ['-', '--', '-.', ':']  # Different linestyles for "less" files
filenames = []

# Loop through all files in the log directory
for i, filename in enumerate(os.listdir(log_dir)):
    if filename.endswith(".log"):
        filenames.append(filename[:-4])

        # Initialize lists for this log file
        rtt_values = []
        snd_cwnd_values = []

        log_file = os.path.join(log_dir, filename)

        # Read data from the log file
        with open(log_file, "r") as file:
            for line in file:
                # Check if the line contains RTT and snd_cwnd values
                if "RTT:" in line and "snd_cwnd:" in line:
                    # Parse the RTT and snd_cwnd values from the line
                    parts = line.split(", ")
                    rtt = int(parts[0].split(":")[1]) /10**3
                    snd_cwnd = int(parts[1].split(":")[1]) /10**6
                    rtt_values.append(rtt)
                    snd_cwnd_values.append(snd_cwnd)

        # Append the data for this log file to the lists
        rtt_list.append(rtt_values)
        snd_cwnd_list.append(snd_cwnd_values)

# Create a single figure for all log files
plt.figure(figsize=(8, 6))

# Plot each log file's data with a different color and linestyle based on filename
for i, (rtt_values, snd_cwnd_values) in enumerate(zip(rtt_list, snd_cwnd_list)):
    color = colors[i % len(colors)]  # Cycle through colors
    linestyle = 'dashed' if "less" not in filenames[i] else 'solid'
    label = filenames[i]  # Set the label as the filename without extension
    plt.scatter(snd_cwnd_values, rtt_values, marker='o', color=color, label=label)
    plt.plot(snd_cwnd_values, rtt_values, linestyle=linestyle, color=color)

plt.xlabel('Sender Congestion Window (MBytes)')  # Updated x-axis label
plt.ylabel('Round-Trip Time (ms)')
plt.title('Delay vs. Sender Congestion Window')
plt.grid(True)
plt.legend()

# Show or save the plot (you can choose to save it if needed)
plt.show()


# Problem3
read json


In [ ]:
import json
import os

log_dir = "problem2"
# Get the list of JSON files in the current directory
# Process each JSON file
for i, filename in enumerate(os.listdir(log_dir)):
    if filename.endswith('.json'):
        log_file = os.path.join(log_dir, filename)
        with open(log_file, 'r') as file:
            try:
                # Load the JSON data
                data = json.load(file)

                # Initialize variables to store results
                best_bps = 0
                best_snd_cwnd_bps = None
                least_rtt = float('inf')
                least_rtt_snd_cwnd = None

                # Iterate through intervals in the JSON data
                for interval in data["intervals"]:
                    for stream in interval["streams"]:
                        bits_per_second = stream["bits_per_second"]
                        snd_cwnd = stream["snd_cwnd"]
                        rtt = stream["rtt"]

                        # Check for the best bits_per_second
                        if bits_per_second > best_bps:
                            best_bps = bits_per_second
                            best_snd_cwnd_bps = snd_cwnd

                        # Check for the least rtt
                        if rtt < least_rtt:
                            least_rtt = rtt
                            least_rtt_snd_cwnd = snd_cwnd

                # Create a log filename by replacing ".json" with ".log"
                log_filename = filename.replace('.json', '.log')

                # Write the results to the log file
                with open(log_filename, 'w') as log_file:
                    log_file.write(f"Best bits_per_second: {best_bps}\n")
                    log_file.write(f"Corresponding snd_cwnd for best bits_per_second: {best_snd_cwnd_bps}\n")
                    log_file.write(f"Least rtt: {least_rtt}\n")
                    log_file.write(f"Corresponding snd_cwnd for least rtt: {least_rtt_snd_cwnd}\n")

                print(f"Processed {filename} and wrote results to {log_filename}")

            except json.JSONDecodeError as e:
                print(f"Error processing {filename}: {e}")


# plot two figure separately

In [ ]:
import os
import matplotlib.pyplot as plt

# Initialize lists to store data and labels
best_bits_per_second_list = []
best_corresponding_snd_cwnd_list = []
least_rtt_list = []
least_corresponding_snd_cwnd_list = []
labels = []  # To store labels

# Loop through all log files in the current directory
log_dir = 'problem3'  # Change this to the directory containing your log files
for filename in os.listdir(log_dir):
    if filename.endswith(".log"):
        labels.append(filename[:-4])  # Append the label based on the filename without extension
        
        with open(os.path.join(log_dir, filename), "r") as file:
            lines = file.readlines()
            for line in lines:
                if line.startswith("Best bits_per_second"):
                    best_bits_per_second = float(line.split(": ")[1]) / 10**6
                elif line.startswith("Corresponding snd_cwnd for best bits_per_second"):
                    best_corresponding_snd_cwnd = int(line.split(": ")[1]) / 10**6
                elif line.startswith("Least rtt"):
                    least_rtt = int(line.split(": ")[1]) / 10**3
                elif line.startswith("Corresponding snd_cwnd for least rtt"):
                    least_corresponding_snd_cwnd = int(line.split(": ")[1]) / 10**6

        # Append data to lists
        best_bits_per_second_list.append(best_bits_per_second)
        best_corresponding_snd_cwnd_list.append(best_corresponding_snd_cwnd)
        least_rtt_list.append(least_rtt)
        least_corresponding_snd_cwnd_list.append(least_corresponding_snd_cwnd)

# Create a single figure for the plots
plt.figure(figsize=(8, 6))

# Plot Best bits_per_second vs. Corresponding snd_cwnd with labels
plt.subplot(2, 1, 1)
for i in range(len(labels)):
    plt.scatter(best_corresponding_snd_cwnd_list[i], best_bits_per_second_list[i], marker='o', label=labels[i])
plt.xlabel('Congestion window size (MBytes)')
plt.ylabel('Best bits_per_second')
plt.title('Best bits_per_second vs. Corresponding snd_cwnd')
plt.legend()

# Plot RTT vs. Corresponding snd_cwnd with labels
plt.subplot(2, 1, 2)
for i in range(len(labels)):
    plt.scatter(least_corresponding_snd_cwnd_list[i], least_rtt_list[i], marker='o', label=labels[i])
plt.xlabel('Congestion window size (MBytes)')
plt.ylabel('RTT (ms)')
plt.title('RTT vs. Corresponding snd_cwnd')
plt.legend()

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plots
plt.show()


## plot problem3

In [37]:
import os
import matplotlib.pyplot as plt

# Initialize lists to store data
best_bits_per_second_list = []
best_corresponding_snd_cwnd_list = []
least_rtt_list = []
least_corresponding_snd_cwnd_list = []
log_filenames = []

# Loop through all files in the current directory
log_dir = 'problem3'  # Change this to the directory containing your log files
for filename in os.listdir(log_dir):
    if filename.endswith(".log"):
        print(f"Processing {filename}")
        log_path = os.path.join(log_dir, filename)
        with open(log_path, "r") as file:
            lines = file.readlines()
            for line in lines:
                if line.startswith("Best bits_per_second"):
                    best_bits_per_second = float(line.split(": ")[1]) / 10**6
                elif line.startswith("Corresponding snd_cwnd for best bits_per_second"):
                    best_corresponding_snd_cwnd = int(line.split(": ")[1]) / 10**6
                elif line.startswith("Least rtt"):
                    least_rtt = int(line.split(": ")[1])
                elif line.startswith("Corresponding snd_cwnd for least rtt"):
                    least_corresponding_snd_cwnd = int(line.split(": ")[1]) / 10**6

        # Append data to lists and store the filename
        best_bits_per_second_list.append(best_bits_per_second)
        best_corresponding_snd_cwnd_list.append(best_corresponding_snd_cwnd)
        least_rtt_list.append(least_rtt)
        least_corresponding_snd_cwnd_list.append(least_corresponding_snd_cwnd)
        log_filenames.append(filename[:-4])  # Remove the '.log' extension and store the filename

# Create a single figure for the plots
fig, ax1 = plt.subplots(figsize=(8, 6))

# Plot RTT vs. Corresponding snd_cwnd on the left y-axis with 'x' marker
ax1.scatter(least_corresponding_snd_cwnd_list, least_rtt_list, marker='x', color='r', label='RTT')
ax1.set_xlabel('congest window size (MBytes)')
ax1.set_ylabel('RTT (ms)', color='r')
ax1.tick_params(axis='y', labelcolor='r')
ax1.set_title('Least RTT and Best bits_per_second vs. congestion window size')

# Create a second y-axis for Best bits_per_second with 'dot' marker
ax2 = ax1.twinx()
ax2.scatter(best_corresponding_snd_cwnd_list, best_bits_per_second_list, marker='.', color='b', label='Best Throughput (Mbits)')
ax2.set_ylabel('Best bits_per_second', color='b')
ax2.tick_params(axis='y', labelcolor='b')

# Add labels for each point
for i, filename in enumerate(log_filenames):
    ax1.annotate(filename, (least_corresponding_snd_cwnd_list[i], least_rtt_list[i]), fontsize=8)
    ax2.annotate(filename, (best_corresponding_snd_cwnd_list[i], best_bits_per_second_list[i]), fontsize=8)

# Add legend
fig.tight_layout()
fig.legend(loc="upper left", bbox_to_anchor=(0.1, 1))

# Show the plot
plt.show()


Processing cubicwireless.log


FileNotFoundError: [Errno 2] No such file or directory: 'cubicwireless.log'